# Installs

In [ ]:
pip install ipdb

# Imports

In [ ]:
import os

import numpy as np
import pandas as pd

# Data prep

In [ ]:
test = pd.read_csv('../input/tweet-sentiment-extraction/test.csv')
test.head()

In [ ]:
train = pd.read_csv('../input/tweet-sentiment-extraction/train.csv')
train.head()

In [ ]:
train.isnull().sum()

In [ ]:
train.dropna(axis = 0, how ='any',inplace=True)

In [ ]:
def jaccard(str1, str2): 
    a = set(str1.lower().split()) 
    b = set(str2.lower().split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

In [ ]:
# Add tokes and counts.
train['text_tokes']   = train.text.str.split()
train['select_tokes'] = train.selected_text.str.split()
train['text_tokes_cnt'] = train.text_tokes.str.len()
train['select_tokes_cnt'] = train.select_tokes.str.len()
train.head(5)

In [ ]:
# Calc jaccard
train['jaccard'] = [jaccard(r.text, r.selected_text) for i,r in train.iterrows()]

In [ ]:
train.jaccard.hist(bins=5)

In [ ]:
# most of the values are within the range of 0.41 from the mean value.
print(train.jaccard.mean())
print(train.jaccard.std())

In [ ]:
# separate out Neutrals and 2 tokes or less: both always resturn self.
print(len(train))
train = train[train.sentiment!='neutral']
print(len(train))
train.sentiment.value_counts()

In [ ]:
# show jaccard after removals
# same .6-.9 thin but hist shift to the left considerably.
train.jaccard.hist(bins=50)

In [ ]:
# This is pretty much the opposite of what a curve ideally should look like for a set of possitive data.
# Ideal is all 1.0 but the curve should go low on left to high in right and should be nice and smooth.
train.jaccard.hist(bins=10)

In [ ]:
# Mean shifte to left and std drops makeing data a bit more regular.
print(train.jaccard.mean())
print(train.jaccard.std())

In [ ]:
# look at Jaccard=0
# These 572 are due mostly to non text character in 'selected' text making jaccard token comparison fail.
# This is such a small percentage of 16k training set its best to drop these all j-0 from training set.
# This however, does not resolve the testing set inacuracies this might cause. 
# Does the testing set have non-word charaters in 'selected'?

pd.options.display.max_colwidth=1000
pd.options.display.max_rows = 500
train[train.jaccard==0.0].sort_values('jaccard', ascending=True)[:5]

In [ ]:
# look at low jaccard to see if you can drop more.
# Yes, at 0.5 drops 1,200 more. So probably drop at or bellow 0.5
cumsum=[]
for jscore in np.arange(0.00, 1.0, 0.01):
    cumsum.append((jscore, len(train[train.jaccard<=jscore].jaccard)))
pd.DataFrame(cumsum).plot(x=0, y=1)

In [ ]:
# zoom in on lower end to see if can drop more than just zeroes.
pd.DataFrame(cumsum)[:10].plot(x=0, y=1)

In [ ]:
pd.options.display.max_colwidth=1000
pd.options.display.max_rows = 500
train[(train.jaccard>=0.0) & (train.jaccard<=0.4)].sort_values('jaccard', ascending=True)[:5]

In [ ]:
len(train[(train.jaccard>0.0) & (train.jaccard<=0.05)])

In [ ]:
train[(train.jaccard>=0.08) & (train.jaccard<=0.09)].sort_values('jaccard', ascending=True)[:50]

# Data - learnings
* The jaccard score curve shows that the majority of the data is crammed into the lower scores making this a very sensitive score
* We will need to somehow expand this spectrum at this lower end to get a more resiliant measure.

In [ ]:
# Next look at example that have low j scores even though they are possitives - try to understrand thei,